In [3]:
import pandas as pd
import json 
import numpy as np
from collections import defaultdict
from glob import glob

# Twitter Data

In [4]:
columns = ["author_id", "id", "created_at", "hashtags",
           "user_mentions", "reply_count", "like_count", "quote_count", "retweet_count"]

In [5]:
df_ls=[]
for file in glob("../data/output/preprocessed/dataframes/*.csv"):
    df = pd.read_csv(file)
    df["author_id"]=df["author_id"].astype('Int64')
    df["id"]=df["id"].astype('Int64')
    df = df[columns]
    df_ls.append(df)

/var/folders/bj/qp6k2wl11h5gpn8j4hg0tgf40000gn/T/ipykernel_35058/3201840456.py:3: DtypeWarning: Columns (0,2,3,5,6,9,10,15,16,17,18,19) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)
/var/folders/bj/qp6k2wl11h5gpn8j4hg0tgf40000gn/T/ipykernel_35058/3201840456.py:3: DtypeWarning: Columns (0,2,3,5,6,7,9,10,15,16,17,18,19) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)
/var/folders/bj/qp6k2wl11h5gpn8j4hg0tgf40000gn/T/ipykernel_35058/3201840456.py:3: DtypeWarning: Columns (0,2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)
/var/folders/bj/qp6k2wl11h5gpn8j4hg0tgf40000gn/T/ipykernel_35058/3201840456.py:3: DtypeWarning: Columns (0,2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)
/var/folders/bj/qp6k2wl11h5gpn8j4hg0tgf40000gn/T/ipykernel_35058/3201840456.py:3: DtypeWarning: Columns (0,2) 

In [6]:
df_merged = pd.concat(df_ls)


In [7]:
len(df_merged)

4819613

In [8]:
df_merged.drop_duplicates(subset=["id"], inplace=True)

In [9]:
len(df_merged)

2771316

In [10]:
columns_=["id", "date", "preprocessed_text",  "wikidata_ids", "event_id", "score", "score_dict"]

In [11]:
df_ls_=[]
for file in glob("../data/events/event_detected_tweets/*.csv"):
    dfs = pd.read_csv(file)
    dfs["id"]=dfs["id"].astype('Int64')
    
    dfs = dfs[columns_]
    dfs = dfs[dfs["score"]>0.3]
    df_ls_.append(dfs)

In [12]:
dfs_merged = pd.concat(df_ls_)

In [13]:
dfs_merged.event_id.value_counts()

6      798911
78     297312
59     211339
97     126225
152     88316
        ...  
90          2
231         2
203         1
180         1
227         1
Name: event_id, Length: 237, dtype: int64

In [65]:
dfs_merged = dfs_merged[dfs_merged["score"]>=0.5]

In [14]:
# join two
df_tweets = pd.merge(dfs_merged, df_merged, on=["id"])

In [16]:
df_tweets.to_csv("../data/final/tweets.csv", index=False)

# Event data

In [21]:
df_event = pd.read_csv("../data/events/event_indexed.csv")

In [22]:
df_event = df_event[["category", "date", "event_text", "main_event", "first_sentence", "first_actor", "wikidata_labels", "wikidata_ids", "event_id"]]

In [25]:
# get the resource wikipedia
urls_sources=[]
for date in df_event.date.tolist():
    splits = date.split("-")
    year, month, day = splits[0], splits[1], splits[2]
    if month=="03":
        month_ ="March"
    
    if month=="02":
        month_ ="February"
    
    day_=int(day)
        
    url = f'https://en.wikipedia.org/wiki/Portal:Current_events/{year}_{month_}_{day_}'
    urls_sources.append(url)

In [27]:
df_event["extractedFrom"]= urls_sources

In [28]:
df_event.to_csv("../data/final/events.csv")

# Merge entities from event and tweets

In [7]:
df_event_entites= pd.read_csv("../data/wikidata/entities_events_detected.csv")
df_tweet_entities = pd.read_csv("../data/wikidata/entities_tweets_detected.csv")

In [8]:
df_entities = pd.concat([df_event_entites,df_tweet_entities])

In [9]:
df_entities = df_entities[["entity_id", "description", "label", "url"]]

In [10]:
df_entities

,entity_id,description,label,url
0,1080137,"military formation size, generally subordinate...",troop,https://www.wikidata.org/wiki/Q1080137
1,1050926,officially designated exclusion area around th...,Chernobyl Exclusion Zone,https://www.wikidata.org/wiki/Q1050926
2,212,sovereign state in Eastern Europe,Ukraine,https://www.wikidata.org/wiki/Q212
3,559549,United States federal service academy,United States Naval Academy,https://www.wikidata.org/wiki/Q559549
4,155868,island in the Black Sea,Snake Island,https://www.wikidata.org/wiki/Q155868
...,...,...,...,...
100256,266232,novel by Arthur Schnitzler,Dream Story,https://www.wikidata.org/wiki/Q266232
100257,18330923,"2014 Swedish, horror-comedy film",American Burger,https://www.wikidata.org/wiki/Q18330923
100258,7161502,NaN,Pell Mell,https://www.wikidata.org/wiki/Q7161502
100259,1267311,trademarked name for a range of condoms,Durex,https://www.wikidata.org/wiki/Q1267311


In [13]:
df_entities.drop_duplicates(subset=["entity_id"], inplace=True)

In [14]:
df_entities.to_csv("../data/final/entities.csv",index=False)